# Criação de data warehouse para dados públicos de atendimentos ambulatoriais do SUS

Esse Jupyter notebook é parte do trabalho de conclusão de curso do MBA em Inteligência Artificial e Big Data, oferecido pelo ICMC - USP, do aluno Danilo Gouvea Silva, da Turma 3.

Parte do capítulo de Avaliação Experimental da monografia, nesse segundo notebook (`sus_dw_eda.ipynb`), após a execução do processo de ETL - "Extract, Transform and Load" - o data warehouse está organizado em 1 tabela de fatos e 12 tabelas de dimensões armazenadas em arquivos Apache Parquet.

Aqui, todas as tabelas serão carregadas e estarão prontras para a análise exploratória através de consultas analíticas. Algumas consultas analíticas serão realizadas como exemplo.

Esse notebook foi criado e utilizado localmente. Para utilizá-lo, é necessário que estejam localmente instalados o Spark, o Java e o Python. Também é recomendado a criação de um ambiente virtual Python para a instalação de todos pacotes de Python necessários, que estão contidos no arquivo de requisitos `requirements.txt`, que disponibilizado junto a esse notebook.

É importante ressaltar que o objetivo desse notebook não é demonstrar, nem guiar a instalação e configuração do Spark numa máquina local.

## Criação da sessão Spark

Nessa seção, é criado uma sessão local de Spark, com apenas um <b>nó mestre</b> e sem <b>nós de trabalho</b> e <b>gerenciador de cluster</b>. As tarefas (<i>tasks</i>) serão executadas pelo <i>driver</i> localizado no nó mestre e utilizarão o máximo de núcleos lógicos de processamento disponíveis no processador local.

In [1]:
# Verifica as versões instaladas de Java, Python e Spark

!java -version
!python --version
!pyspark --version

java version "1.8.0_411"
Java(TM) SE Runtime Environment (build 1.8.0_411-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.411-b09, mixed mode)


Python 3.10.5


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, Java HotSpot(TM) 64-Bit Server VM, 1.8.0_381
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


In [2]:
# Configura corretamente as variáveis de ambiente do Spark

!pip install findspark
import findspark
findspark.init()

In [3]:
# Cria a sessão de Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('sus_dw_eda') \
    .master('local[*]') \
    .getOrCreate()

spark

## Carregamento das tabelas de fatos e dimensões 

In [4]:
# Importa as funções e tipos de dados do Spark

import pyspark.sql.functions as F
import pyspark.sql.types as T

### Carregamento da tabela de fatos SIASUS - SERVIÇO DE INFORMAÇÕES AMBULATORIAIS DO SUS (`sia_df`)

In [5]:
# Carrega a tabela de fatos SIA - Serviço de Informações Ambulatoriais

sia_df = spark.read.parquet('sus-data-warehouse/sia')

In [6]:
# Conta o número total de registros na tabela de fatos

sia_df_count = sia_df.count()

print(f'Número de registros: {sia_df_count:,}')

Número de registros: 1,742,743,969


In [7]:
# Exibe a lista de colunas do dataframe SIA

sia_df \
    .printSchema()

root
 |-- PA_CMP: string (nullable = true)
 |-- PA_CODUNI: string (nullable = true)
 |-- PA_TPUPS: string (nullable = true)
 |-- PA_UFMUN: string (nullable = true)
 |-- PA_PROC_ID: string (nullable = true)
 |-- PA_DOCORIG: string (nullable = true)
 |-- PA_CNSMED: string (nullable = true)
 |-- PA_CBOCOD: string (nullable = true)
 |-- PA_MOTSAI: string (nullable = true)
 |-- PA_CIDPRI: string (nullable = true)
 |-- PA_CIDSEC: string (nullable = true)
 |-- PA_CIDCAS: string (nullable = true)
 |-- PA_CATEND: string (nullable = true)
 |-- PA_IDADE: integer (nullable = true)
 |-- PA_SEXO: string (nullable = true)
 |-- PA_RACACOR: string (nullable = true)
 |-- PA_MUNPCN: string (nullable = true)
 |-- PA_QTDAPR: integer (nullable = true)
 |-- PA_VALAPR: float (nullable = true)



### Carregamento das tabelas de dimensões

Nessa seção, serão carregadas as tabelas de dimensões:
- `municipios_df`: listagem do IBGE de todos os munícipios brasileiros, estados, regiões e outras informações; `sia_df['PA_UFMUN]`
- `cnes_df`: Cadastro Nacional de Estabelecimentos de Saúde; `sia_df['PA_CODUNI']`
- `sigtap_proced_df`: listagem dos Procedimentos oferecidos pelo SUS; `sia_df['PA_PROC_ID']`
- `cid_df`: CID-10 Código Internacional de Doenças; `sia_df['PA_CIDPRI', 'PA_CIDSEC', 'PA_CIDCAS']`
- `ano_mes_df`: Dimensão "data"no formato AAAAMM; `sia_df['PA_CMP']`
- `cbocod_df`: Código Brasileiro de Ocupações; `sia_df['PA_CBOCOD']`
- `tpups_df`: Tipos de Estabelcimentos de Saúde; `sia_df['PA_TPUPS']`
- `catend_df`: Caráter de Atendimento; `sia_df['PA_CATEND']`
- `docorig_df`: Tipo de Documento de Origem da produção ambulatorial; `sia_df['PA_DOCORIG']`
- `sexo_df`: Sexo do paciente; `sia_df['PA_SEXO']`
- `raca_cor_df`: Raça/Cor do paciente; `sia_df['PA_RACACOR']`
- `motsai_df`: Motivos de saída. `sia_df['PA_MOTSAI']`

In [8]:
# Carrega as tabelas de dimensões

municipios_df = spark.read.parquet('sus-data-warehouse/municipios')
cnes_df = spark.read.parquet('sus-data-warehouse/cnes')
sigtap_proced_df = spark.read.parquet('sus-data-warehouse/sigtap_proced')
cid_df = spark.read.parquet('sus-data-warehouse/cid')
ano_mes_df = spark.read.parquet('sus-data-warehouse/ano_mes')
cbocod_df = spark.read.parquet('sus-data-warehouse/cbocod')
tpups_df = spark.read.parquet('sus-data-warehouse/tpups')
catend_df = spark.read.parquet('sus-data-warehouse/catend')
docorig_df = spark.read.parquet('sus-data-warehouse/docorig')
sexo_df = spark.read.parquet('sus-data-warehouse/sexo')
raca_cor_df = spark.read.parquet('sus-data-warehouse/raca_cor')
motsai_df = spark.read.parquet('sus-data-warehouse/motsai')


In [9]:
# Verifica os 'schemas' das tabelas de dimensões

print('MUNICIPIOS:')
municipios_df.printSchema()

print('CNES:')
cnes_df.printSchema()

print('SIGTAP_PROCED:')
sigtap_proced_df.printSchema()

print('CID:')
cid_df.printSchema()

print('ANO_MES:')
ano_mes_df.printSchema()

print('CBOCOD:')
cbocod_df.printSchema()

print('TPUPS:')
tpups_df.printSchema()

print('CATEND:')
catend_df.printSchema()

print('DOCORIG:')
docorig_df.printSchema()

print('SEXO:')
sexo_df.printSchema()

print('RACA_COR:')
raca_cor_df.printSchema()

print('MOTSAI:')
motsai_df.printSchema()

MUNICIPIOS:
root
 |-- PA_UFMUN: string (nullable = true)
 |-- NOME: string (nullable = true)
 |-- MICRORREGIAO: string (nullable = true)
 |-- MESORREGIAO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- UF_NOME: string (nullable = true)
 |-- REGIAO_SIGLA: string (nullable = true)
 |-- REGIAO: string (nullable = true)
 |-- REGIAO_IMEDIATA: string (nullable = true)
 |-- REGIAO_INTERMEDIARIA: string (nullable = true)

CNES:
root
 |-- CO_TIPO_ESTABELECIMENTO: string (nullable = true)
 |-- TP_UNIDADE: string (nullable = true)
 |-- CO_UNIDADE: string (nullable = true)
 |-- PA_CODUNI: string (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: string (nullable = true)
 |-- TP_PFPJ: string (nullable = true)
 |-- NIVEL_DEP: string (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: str

## Exemplos de Consultas Analíticas
O objetivo dessa seção é mostrar alguns exemplos de utilização do modelo dimensional do nosso data warehouse.

Aqui, serão realizadas algumas consultas respondendo perguntas hipotéticas sobre a tabela de fatos.

### Psicoterapia em Unidades Básicas de Saúde para recorte populacional e geográfico específico
Qual é a quantidade de procedimentos e valores aprovados pelo SUS para procedimentos **Psicoterápicos**, durantes os anos de 2018 a 2020, realizados em Unidades Básicas de Saúde, em pacientes Pretos e do sexo Masculino, agrupados por Região do País, Ano e Categoria do CID Primário?

In [10]:
sia_df \
    .join(municipios_df.select('PA_UFMUN', 'REGIAO_SIGLA'), on='PA_UFMUN') \
    .join(ano_mes_df.select('PA_CMP', 'ANO'), on='PA_CMP') \
    .join(cnes_df.select('PA_CODUNI', 'TP_UNIDADE_DESC'), on='PA_CODUNI') \
    .join(sigtap_proced_df.select('PA_PROC_ID', 'NO_PROCEDIMENTO', 'DT_COMPETENCIA'), on=[sia_df['PA_PROC_ID'] == sigtap_proced_df['PA_PROC_ID'], 
                                                                                          sia_df['PA_CMP'] == sigtap_proced_df['DT_COMPETENCIA']]) \
    .join(cid_df.select('CO_CID', 'NO_CID', 'CO_CATEG', 'NO_CATEG'), on=sia_df['PA_CIDPRI'] == cid_df['CO_CID']) \
    .join(sexo_df, on='PA_SEXO') \
    .join(raca_cor_df, on='PA_RACACOR') \
    .where('TP_UNIDADE_DESC LIKE "%UNIDADE BASICA%"') \
    .where('NO_PROCEDIMENTO LIKE "%PSICOTERAPIA%"') \
    .where('ANO in ("2018", "2019", "2020")') \
    .where('SEXO_DESC = "Masculino"') \
    .where('RACACOR_DESC = "PRETA"') \
    .select('REGIAO_SIGLA', 'ANO', 'CO_CATEG', 'NO_CATEG', 'PA_QTDAPR', 'PA_VALAPR' ) \
    .groupBy('REGIAO_SIGLA', 'ANO', 'CO_CATEG', 'NO_CATEG' ) \
    .agg({'PA_QTDAPR':'sum', 'PA_VALAPR':'sum'}) \
    .sort('REGIAO_SIGLA', 'ANO', 'sum(PA_VALAPR)', ascending=[True, True, False]) \
    .withColumn('sum(PA_VALAPR)', F.format_number('sum(PA_VALAPR)', 2)) \
    .withColumnRenamed('sum(PA_QTDAPR)', 'PROCEDIMENTOS') \
    .withColumnRenamed('sum(PA_VALAPR)', 'VALOR') \
    .show(60, truncate=False)

+------------+----+--------+--------------------------------------------------------------+-------------+------+
|REGIAO_SIGLA|ANO |CO_CATEG|NO_CATEG                                                      |PROCEDIMENTOS|VALOR |
+------------+----+--------+--------------------------------------------------------------+-------------+------+
|NE          |2018|F20     |Esquizofrenia                                                 |86           |219.30|
|NE          |2018|F32     |Episodios depressivos                                         |13           |33.15 |
|NE          |2019|F20     |Esquizofrenia                                                 |13           |33.15 |
|NE          |2019|F41     |Transtornos ansiosos                                          |1            |2.55  |
|NE          |2019|F91     |Disturbios de conduta                                         |1            |2.55  |
|NE          |2020|F60     |Transtornos especificos da personalidade                      |16   

### Procedimentos e Doenças que mais gastam recursos do SUS na cidade de Salto-SP em atendimentos ambulatoriais

Quais procedimentos mais demandaram recursos financeiros do SUS na cidade de Salto-SP no ano de 2020?

In [11]:
sia_df \
    .join(municipios_df.select('PA_UFMUN', 'NOME', 'UF'), on='PA_UFMUN') \
    .join(ano_mes_df.select('PA_CMP', 'ANO'), on='PA_CMP') \
    .join(cid_df.select('CO_CID', 'NO_CID', 'CO_CATEG', 'NO_CATEG'), on=sia_df['PA_CIDPRI'] == cid_df['CO_CID']) \
    .join(sigtap_proced_df.select('PA_PROC_ID', 'NO_PROCEDIMENTO', 'DT_COMPETENCIA'), on=[sia_df['PA_PROC_ID'] == sigtap_proced_df['PA_PROC_ID'], 
                                                                                          sia_df['PA_CMP'] == sigtap_proced_df['DT_COMPETENCIA']]) \
    .where('ANO = "2020"') \
    .where('NOME ILIKE "SALTO" AND UF = "SP"') \
    .select('NO_PROCEDIMENTO', 'PA_QTDAPR', 'PA_VALAPR') \
    .groupBy('NO_PROCEDIMENTO') \
    .agg({'PA_QTDAPR':'sum', 'PA_VALAPR':'sum'}) \
    .sort('sum(PA_VALAPR)', ascending=False) \
    .withColumn('sum(PA_VALAPR)', F.format_number('sum(PA_VALAPR)', 2)) \
    .withColumnRenamed('sum(PA_QTDAPR)', 'PROCEDIMENTOS') \
    .withColumnRenamed('sum(PA_VALAPR)', 'VALOR') \
    .show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-------------+----------+
|NO_PROCEDIMENTO                                                                                                     |PROCEDIMENTOS|VALOR     |
+--------------------------------------------------------------------------------------------------------------------+-------------+----------+
|TOMOGRAFIA COMPUTADORIZADA DE TORAX                                                                                 |1750         |238,717.51|
|ATENDIMENTO FISIOTERAPÊUTICO NAS ALTERAÇÕES MOTORAS                                                                 |20799        |97,131.33 |
|TOMOGRAFIA COMPUTADORIZADA DO CRANIO                                                                                |585          |57,002.40 |
|EXAME ANATOMO-PATOLÓGICO PARA CONGELAMENTO / PARAFINA POR PEÇA CIRURGICA OU POR BIOPSIA (EXCETO COLO UTERINO E MAMA)|1686         |40,4

De acordo com o resultado da consulta acima, é possível perceber que procedimentos de Tomografia Computadorizada são os que mais demandaram recursos financeiros do SUS para Salto-SP, em 2020.

Quais CIDs (Classificação Internacional de Doenças) levam à realização de procedimentos de tomografica computadorizada em Salto-SP? A consulta abaixo responde essa pergunta.

In [12]:
sia_df \
    .join(municipios_df.select('PA_UFMUN', 'NOME', 'UF'), on='PA_UFMUN') \
    .join(ano_mes_df.select('PA_CMP', 'ANO'), on='PA_CMP') \
    .join(cid_df.select('CO_CID', 'NO_CID', 'CO_CATEG', 'NO_CATEG'), on=sia_df['PA_CIDPRI'] == cid_df['CO_CID']) \
    .join(sigtap_proced_df.select('PA_PROC_ID', 'NO_PROCEDIMENTO', 'DT_COMPETENCIA'), on=[sia_df['PA_PROC_ID'] == sigtap_proced_df['PA_PROC_ID'], 
                                                                                          sia_df['PA_CMP'] == sigtap_proced_df['DT_COMPETENCIA']]) \
    .where('ANO = "2020"') \
    .where('NOME ILIKE "salto" AND UF = "SP"') \
    .where('NO_PROCEDIMENTO ILIKE "%tomografia%computadorizada%"') \
    .select('CO_CATEG', 'CO_CID', 'NO_CID', 'PA_QTDAPR', 'PA_VALAPR') \
    .groupBy('CO_CATEG', 'CO_CID', 'NO_CID') \
    .agg({'PA_QTDAPR':'sum', 'PA_VALAPR':'sum'}) \
    .sort('sum(PA_VALAPR)', ascending=False) \
    .withColumn('sum(PA_VALAPR)', F.format_number('sum(PA_VALAPR)', 2)) \
    .withColumnRenamed('sum(PA_QTDAPR)', 'PROCEDIMENTOS') \
    .withColumnRenamed('sum(PA_VALAPR)', 'VALOR') \
    .show(truncate=False)

+--------+------+--------------------------------------------------------------------------------+-------------+----------+
|CO_CATEG|CO_CID|NO_CID                                                                          |PROCEDIMENTOS|VALOR     |
+--------+------+--------------------------------------------------------------------------------+-------------+----------+
|R52     |R529  |Dor não especificada                                                            |2899         |366,800.78|
|R10     |R104  |Outras dores abdominais e as não especificadas                                  |59           |7,980.39  |
|I64     |I64   |Acidente vascular cerebral, não especificado como hemorrágico ou isquêmico      |74           |7,210.56  |
|M79     |M796  |Dor em membro                                                                   |17           |1,683.35  |
|M51     |M510  |Transtornos de discos lombares e de outros discos intervertebrais com mielopatia|1            |101.10    |
|S52    

Em segundo lugar, o procedimento **Atendimento fisioterapêutico nas alterações motoras** também gasta recursos consideráveis do SUS em Salto-SP.
Quais CIDs fazem uso desse procedimento?
A consulta abaixo responde a essa pergunta.

In [13]:
sia_df \
    .join(municipios_df.select('PA_UFMUN', 'NOME', 'UF'), on='PA_UFMUN') \
    .join(ano_mes_df.select('PA_CMP', 'ANO'), on='PA_CMP') \
    .join(cid_df.select('CO_CID', 'NO_CID', 'CO_CATEG', 'NO_CATEG'), on=sia_df['PA_CIDPRI'] == cid_df['CO_CID']) \
    .join(sigtap_proced_df.select('PA_PROC_ID', 'NO_PROCEDIMENTO', 'DT_COMPETENCIA'), on=[sia_df['PA_PROC_ID'] == sigtap_proced_df['PA_PROC_ID'], 
                                                                                          sia_df['PA_CMP'] == sigtap_proced_df['DT_COMPETENCIA']]) \
    .where('ANO = "2020"') \
    .where('NOME ILIKE "salto" AND UF = "SP"') \
    .where('NO_PROCEDIMENTO ILIKE "%ATENDIMENTO FISIOTERAPÊUTICO NAS ALTERAÇÕES MOTORAS%"') \
    .select('CO_CATEG', 'CO_CID', 'NO_CID', 'PA_QTDAPR', 'PA_VALAPR') \
    .groupBy('CO_CATEG', 'CO_CID', 'NO_CID') \
    .agg({'PA_QTDAPR':'sum', 'PA_VALAPR':'sum'}) \
    .sort('sum(PA_VALAPR)', ascending=False) \
    .withColumn('sum(PA_VALAPR)', F.format_number('sum(PA_VALAPR)', 2)) \
    .withColumnRenamed('sum(PA_QTDAPR)', 'PROCEDIMENTOS') \
    .withColumnRenamed('sum(PA_VALAPR)', 'VALOR') \
    .show(truncate=False)

+--------+------+----------------------------------------------------------+-------------+---------+
|CO_CATEG|CO_CID|NO_CID                                                    |PROCEDIMENTOS|VALOR    |
+--------+------+----------------------------------------------------------+-------------+---------+
|M96     |M969  |Transtorno osteomuscular não especificado pós-procedimento|10650        |49,735.50|
|M54     |M544  |Lumbago com ciática                                       |8929         |41,698.43|
|M25     |M255  |Dor articular                                             |861          |4,020.87 |
|M65     |M654  |Tenossinovite estilóide radial [de quervain]              |296          |1,382.32 |
|G81     |G811  |Hemiplegia espástica                                      |25           |116.75   |
|M65     |M659  |Sinovite e tenossinovite não especificadas                |19           |88.73    |
|M25     |M256  |Rigidez articular não classificada em outra parte         |18           |8

Qual é o perfil dos pacientes que mais utilizaram o procedimento **Atendimento fisioterapêutico nas alterações motoras** em Salto-SP em 2020?

In [14]:
sia_df \
    .join(municipios_df.select('PA_UFMUN', 'NOME', 'UF'), on='PA_UFMUN') \
    .join(ano_mes_df.select('PA_CMP', 'ANO'), on='PA_CMP') \
    .join(sigtap_proced_df.select('PA_PROC_ID', 'NO_PROCEDIMENTO', 'DT_COMPETENCIA'), on=[sia_df['PA_PROC_ID'] == sigtap_proced_df['PA_PROC_ID'], 
                                                                                          sia_df['PA_CMP'] == sigtap_proced_df['DT_COMPETENCIA']]) \
    .join(sexo_df, on='PA_SEXO') \
    .join(raca_cor_df, on='PA_RACACOR') \
    .where('ANO = "2020"') \
    .where('NOME ILIKE "salto" AND UF = "SP"') \
    .where('NO_PROCEDIMENTO ILIKE "%ATENDIMENTO FISIOTERAPÊUTICO NAS ALTERAÇÕES MOTORAS%"') \
    .select('SEXO_DESC', 'RACACOR_DESC', 'PA_IDADE', 'PA_QTDAPR', 'PA_VALAPR') \
    .groupBy('SEXO_DESC', 'RACACOR_DESC') \
    .agg({'PA_IDADE':'avg', 'PA_QTDAPR':'sum', 'PA_VALAPR':'sum'}) \
    .sort('sum(PA_VALAPR)', ascending=False) \
    .withColumn('sum(PA_VALAPR)', F.format_number('sum(PA_VALAPR)', 2)) \
    .withColumn('avg(PA_IDADE)', F.round('avg(PA_IDADE)', 1)) \
    .withColumnRenamed('avg(PA_IDADE)', 'IDADE_MEDIA') \
    .withColumnRenamed('sum(PA_QTDAPR)', 'PROCEDIMENTOS') \
    .withColumnRenamed('sum(PA_VALAPR)', 'VALOR') \
    .show(truncate=False)

+---------+--------------+-------------+---------+-----------+
|SEXO_DESC|RACACOR_DESC  |PROCEDIMENTOS|VALOR    |IDADE_MEDIA|
+---------+--------------+-------------+---------+-----------+
|Feminino |BRANCA        |9429         |44,033.43|53.9       |
|Masculino|BRANCA        |5857         |27,352.19|47.2       |
|Feminino |PARDA         |2208         |10,311.36|51.8       |
|Masculino|PARDA         |1588         |7,415.96 |47.8       |
|Feminino |PRETA         |1011         |4,721.37 |54.8       |
|Masculino|PRETA         |606          |2,830.02 |47.3       |
|Feminino |AMARELA       |61           |284.87   |48.4       |
|Masculino|SEM INFORMAÇÃO|26           |121.42   |55.5       |
|Feminino |SEM INFORMAÇÃO|12           |56.04    |58.3       |
|Masculino|AMARELA       |1            |4.67     |59.0       |
+---------+--------------+-------------+---------+-----------+



## Encerramento da sessão Spark

In [15]:
spark.stop()